In [1]:
import random
import string
pw = random_string = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(15))
print(pw)

dAt86qAlUM1FTrb


In [6]:
import json

# 定義多組物品的編號、pk和sk的值
items = [
    {"id": "IOT1234567", 
     "pk": "010010010100111101010100001100010011001000110011001101000011010100110110001110111001100011010111011011110011101110011001100010001100110000111011100110010111010101110011", "sk": "110001101001111100110110110111101110",
     "sk":"110001101001111100110110110111101110"
     }
]

# 指定文件名稱
filename = "items.json"

# 將多組數據寫入JSON文件
with open(filename, "w") as file:
    json.dump(items, file, indent=4)

print(f"物品數據已保存到 {filename} 文件中")

物品數據已保存到 items.json 文件中


In [7]:
# 初始化變數
items = []

# 從JSON文件中讀取數據
with open(filename, "r") as file:
    items = json.load(file)

# 定義要查找的物品編號
target_id = "IOT1234561"
target_pk = None
target_sk = None

# 查找對應編號的物品pk
for item in items:
    if item["id"] == target_id:
        target_pk = item["pk"]
        target_sk = item["sk"]
        break

if target_pk:
    print(f"編號為 {target_id} 的物品pk: {target_pk}")
else:
    print(f"找不到編號為 {target_id} 的物品")

找不到編號為 IOT1234561 的物品
